# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235298541598                   -0.50    7.0
  2   -7.250426228169       -1.82       -1.40    1.0
  3   -7.251325573995       -3.05       -2.19    3.0
  4   -7.251319800801   +   -5.24       -2.47    2.0
  5   -7.251331206024       -4.94       -2.72    2.0
  6   -7.251338184667       -5.16       -3.23    2.0
  7   -7.251338790340       -6.22       -4.12    1.0
  8   -7.251338796240       -8.23       -4.38    4.0
  9   -7.251338798623       -8.62       -5.07    3.0
 10   -7.251338798702      -10.10       -5.85    3.0
 11   -7.251338798704      -11.89       -6.09    4.0
 12   -7.251338798705      -12.05       -6.83    2.0
 13   -7.251338798705      -13.57       -7.15    4.0
 14   -7.251338798705      -14.75       -7.58    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.055339624058469414
[ Info: Arnoldi iteration step 2: normres = 0.5060028727802615
[ Info: Arnoldi iteration step 3: normres = 0.9170961668717856
[ Info: Arnoldi iteration step 4: normres = 0.27247679521457113
[ Info: Arnoldi iteration step 5: normres = 0.5869237108595587
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.90e-02, 8.75e-02, 5.43e-01, 2.01e-01, 1.11e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2251501640088946
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (7.02e-03, 1.20e-01, 1.29e-01, 1.13e-01, 1.71e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09202073499483321
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.09e-04, 9.79e-03, 1.02e-02, 5.76e-02, 6.82e-02)
[ Info: Arnoldi iteration step 8: normres = 0.07615495993703225
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.00e